## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-17-10-06-36 +0000,bbc,Five young people die in Spain apartment block...,https://www.bbc.com/news/articles/c99jngpymyjo...
1,2026-02-17-10-04-08 +0000,bbc,Jesse Jackson: Veteran civil rights leader who...,https://www.bbc.com/news/articles/cx77214zz9eo...
2,2026-02-17-10-03-10 +0000,nyt,The Drone Games: Flying Cameras Are Everywhere...,https://www.nytimes.com/2026/02/17/world/europ...
3,2026-02-17-10-03-02 +0000,nyt,An M.L.K. Landmark Is Revived at a Tense Momen...,https://www.nytimes.com/2026/02/17/us/martin-l...
4,2026-02-17-10-03-00 +0000,nyt,Trump Sought Vast Budget Cuts. Congress Grante...,https://www.nytimes.com/2026/02/17/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2562/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,29
42,ice,11
54,new,11
243,will,11
85,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2026-02-17-09-25-31 +0000,nyt,Trump Said He’d End the War in Ukraine in a Da...,https://www.nytimes.com/2026/02/17/world/europ...,68
34,2026-02-17-07-59-55 +0000,bbc,Hillary Clinton accuses Trump administration o...,https://www.bbc.com/news/articles/czx47k34yqxo...,63
11,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...,61
173,2026-02-16-18-07-29 +0000,wapo,"Popular anger burns in Iran after crackdown, a...",https://www.washingtonpost.com/world/2026/02/1...,60
166,2026-02-16-18-29-22 +0000,nypost,Trump says Board of Peace will discuss how to ...,https://nypost.com/2026/02/16/us-news/trump-sa...,59


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,68,2026-02-17-09-25-31 +0000,nyt,Trump Said He’d End the War in Ukraine in a Da...,https://www.nytimes.com/2026/02/17/world/europ...
86,52,2026-02-17-00-51-56 +0000,nypost,Woman claims ‘my father was the shooter’ who k...,https://nypost.com/2026/02/16/us-news/woman-cl...
62,46,2026-02-17-02-44-32 +0000,nypost,Mayor Karen Bass calls on LA Olympics boss Cas...,https://nypost.com/2026/02/16/us-news/mayor-ka...
11,42,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...
92,37,2026-02-17-00-12-58 +0000,bbc,The tech bros might show more humility in Delh...,https://www.bbc.com/news/articles/cr5l6gnen72o...
40,36,2026-02-17-06-29-35 +0000,nyt,U.S. and Iran Gear Up for Nuclear Talks Amid R...,https://www.nytimes.com/2026/02/17/us/politics...
101,35,2026-02-16-23-37-30 +0000,bbc,'It's never too late': Savannah Guthrie's new ...,https://www.bbc.com/news/articles/c5yrv2g16e2o...
236,34,2026-02-16-13-44-07 +0000,cbc,"Dana Eden, co-creator of Israeli TV series Teh...",https://www.cbc.ca/news/world/dana-eden-israel...
122,34,2026-02-16-22-31-41 +0000,nypost,Former top Letitia James aide helps funnel mon...,https://nypost.com/2026/02/16/us-news/former-t...
213,28,2026-02-16-15-27-46 +0000,nypost,Obama walks back casual claim aliens are ‘real...,https://nypost.com/2026/02/16/us-news/barack-o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
